# Lookout Audit & Insider Threat Hunting

This notebook provides threat hunting queries for investigating administrative actions and potential insider threats.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- AUDIT events ingested

## 1. Administrative Actions Overview

Review all administrative actions in the Lookout console.

In [1]:
LookoutEvents
| where EventType == "AUDIT"
| summarize 
    ActionCount = count(),
    ActionTypes = make_set(AuditType)
    by ActorGuid, ActorType
| sort by ActionCount desc

ActorGuid,ActorType,ActionCount,ActionTypes
admin-001-guid-abcd-1234,ADMIN_USER,156,"[""LOGIN"", ""POLICY_UPDATE"", ""USER_CREATE"", ""CONFIG_CHANGE""]"
admin-002-guid-efgh-5678,ADMIN_USER,89,"[""LOGIN"", ""DEVICE_DEACTIVATE"", ""REPORT_GENERATE""]"
system-001-guid-ijkl-9012,SYSTEM,2450,"[""AUTO_SCAN"", ""THREAT_DETECT"", ""SYNC""]"
admin-003-guid-mnop-3456,ADMIN_USER,45,"[""LOGIN"", ""USER_DELETE"", ""POLICY_VIEW""]"
api-001-guid-qrst-7890,API_SERVICE,1823,"[""DATA_EXPORT"", ""SYNC"", ""QUERY""]"


## 2. Policy Changes

Track changes to security policies that could weaken defenses.

In [2]:
LookoutEvents
| where EventType == "AUDIT"
| where AuditType has_any ("POLICY", "CONFIG", "SETTING")
| project 
    TimeGenerated,
    ActorGuid,
    ActorType,
    AuditType,
    AuditAttributeChanges,
    TargetType,
    TargetGuid
| sort by TimeGenerated desc

TimeGenerated,ActorGuid,ActorType,AuditType,AuditAttributeChanges,TargetType,TargetGuid
2025-01-05T14:30:00Z,admin-001-guid-abcd-1234,ADMIN_USER,POLICY_UPDATE,"{""malwareDetection"": ""enabled->disabled""}",SECURITY_POLICY,policy-mobile-001
2025-01-05T11:15:00Z,admin-001-guid-abcd-1234,ADMIN_USER,CONFIG_CHANGE,"{""alertThreshold"": ""HIGH->CRITICAL""}",ALERT_CONFIG,config-alert-001
2025-01-04T16:45:00Z,admin-002-guid-efgh-5678,ADMIN_USER,POLICY_UPDATE,"{""complianceCheck"": ""strict->relaxed""}",COMPLIANCE_POLICY,policy-compliance-002
2025-01-04T09:30:00Z,admin-001-guid-abcd-1234,ADMIN_USER,SETTING_CHANGE,"{""dataRetention"": ""90days->30days""}",SYSTEM_SETTING,setting-retention-001
2025-01-03T13:00:00Z,admin-003-guid-mnop-3456,ADMIN_USER,POLICY_CREATE,"{""name"": ""New BYOD Policy""}",SECURITY_POLICY,policy-byod-003


## 3. Unusual Admin Activity Patterns

Identify administrators with unusual activity volumes.

In [3]:
LookoutEvents
| where EventType == "AUDIT"
| where ActorType in ("ADMIN_USER", "USER")
| summarize 
    DailyActions = count(),
    ActionTypes = make_set(AuditType)
    by ActorGuid, bin(TimeGenerated, 1d)
| where DailyActions > 10
| sort by DailyActions desc

ActorGuid,TimeGenerated,DailyActions,ActionTypes
admin-001-guid-abcd-1234,2025-01-05T00:00:00Z,47,"[""LOGIN"", ""POLICY_UPDATE"", ""USER_DELETE"", ""CONFIG_CHANGE"", ""DEVICE_DEACTIVATE""]"
admin-001-guid-abcd-1234,2025-01-04T00:00:00Z,32,"[""LOGIN"", ""POLICY_UPDATE"", ""SETTING_CHANGE""]"
admin-002-guid-efgh-5678,2025-01-05T00:00:00Z,28,"[""LOGIN"", ""DEVICE_DEACTIVATE"", ""REPORT_GENERATE"", ""USER_VIEW""]"
admin-003-guid-mnop-3456,2025-01-03T00:00:00Z,18,"[""LOGIN"", ""POLICY_CREATE"", ""USER_CREATE""]"
admin-002-guid-efgh-5678,2025-01-02T00:00:00Z,15,"[""LOGIN"", ""REPORT_GENERATE""]"


## 4. Device Deactivations

Monitor device deactivation events that could indicate cleanup of compromised devices.

In [4]:
LookoutEvents
| where EventType == "AUDIT"
| where AuditType has_any ("DEACTIVATE", "DELETE", "REMOVE")
| project 
    TimeGenerated,
    ActorGuid,
    ActorType,
    AuditType,
    TargetType,
    TargetGuid,
    TargetEmailAddress
| sort by TimeGenerated desc

TimeGenerated,ActorGuid,ActorType,AuditType,TargetType,TargetGuid,TargetEmailAddress
2025-01-05T15:45:00Z,admin-001-guid-abcd-1234,ADMIN_USER,DEVICE_DEACTIVATE,DEVICE,device-001-compromised,john.smith@contoso.com
2025-01-05T14:20:00Z,admin-002-guid-efgh-5678,ADMIN_USER,DEVICE_DELETE,DEVICE,device-002-lost,sarah.jones@contoso.com
2025-01-05T11:30:00Z,admin-001-guid-abcd-1234,ADMIN_USER,USER_REMOVE,USER,user-former-employee,former.employee@contoso.com
2025-01-04T16:00:00Z,admin-002-guid-efgh-5678,ADMIN_USER,DEVICE_DEACTIVATE,DEVICE,device-003-old,retired.user@contoso.com
2025-01-04T10:15:00Z,admin-003-guid-mnop-3456,ADMIN_USER,DEVICE_DELETE,DEVICE,device-004-replaced,mike.wilson@contoso.com


## 5. Audit Activity Timeline

Visualize administrative activity over time.

In [5]:
LookoutEvents
| where EventType == "AUDIT"
| summarize Count = count() by bin(TimeGenerated, 1h), AuditType
| render timechart

Time Chart rendered - Audit activity trends showing LOGIN, POLICY_UPDATE, and CONFIG_CHANGE events over time

## 6. System vs User Actions

Compare automated system actions versus manual user actions.

In [6]:
LookoutEvents
| where EventType == "AUDIT"
| summarize Count = count() by ActorType
| render piechart

Pie Chart rendered - Distribution of actions showing SYSTEM (55%), ADMIN_USER (28%), and API_SERVICE (17%)